BERTopic Demo
https://maartengr.github.io/BERTopic/

In [1]:
%pip install bertopic
%pip install bertopic[flair, gensim, spacy, use]
%pip install openai

Note: you may need to restart the kernel to use updated packages.
ERROR: Invalid requirement: 'bertopic[flair,'
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Fetch texts from db and compute topics with BERTopic

In [4]:
from bertopic import BERTopic
from embedding import fetch_entries
db_name = "law_database.db"
texts, uuids = fetch_entries(db_name)

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(texts)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [1]:
from bertopic import BERTopic
from embedding import fetch_entries_with_embeddings
db_name = "law_database.db"
emedding_model = "all-MiniLM-L6-v2"
texts, uuids, embeddings = fetch_entries_with_embeddings(db_name)
topic_model = BERTopic(embedding_model=emedding_model)
topics, probs = topic_model.fit_transform(texts, embeddings=embeddings)

[nltk_data] Downloading package punkt to /home/luc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


hi dad
hi mom


In [15]:
# topic_model.save("./topic_model", serialization="safetensors", save_ctfidf=True, save_embedding_model=emedding_model)
topic_model.save("temp", serialization="pickle")

2024-02-14 15:04:50,580 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [2]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
topic_model = BERTopic.load("temp")

Insert the topics into the db

In [4]:
from embedding import insert_label
f = topic_model.get_topic_info()
t = f['Name']
topic_names = t.tolist()
print(topic_names)
for topic_name in topic_names:
    insert_label(db_name, topic_name)

['-1_optometry_optometrist_optometric_practitioner', '0_nursing_degree_nurses_baccalaureate', '1_supervision_naturopathic_supervising_supervise', '2_misconduct_unprofessional_discretion_license', '3_residential_residence_facilities_facility', '4_intervention_disciplinary_registered_termination', '5_nursing_requirements_academic_postsecondary', '6_nursing_nurse_committees_committee', '7_optometry_optometrist_ophthalmologist_lenses', '8_compliance_certification_licensees_nursing', '9_renewal_renew_license_renewed', '10_residential_residence_facilities_facility', '11_midwife_midwives_nurse_standardized', '12_committee_registered_appointment_appointed', '13_psychopharmacology_pharmacology_psychologists_psychologist', '14_abortion_midwifery_competency_procedure', '15_optometry_regulations_optometric_optometrists', '16_optometry_optometric_license_optometrist', '17_license_unprofessional_licensure_drug', '18_nursing_enroll_degree_institution', '19_licensure_fee_fees_nursing', '20_postpartum_

In [5]:
topic_model.get_topic(0)

[['nursing', 0.4826834201812744],
 ['degree', 0.3612825870513916],
 ['nurses', 0.3525093197822571],
 ['baccalaureate', 0.3432251811027527],
 ['nurse', 0.329161673784256],
 ['institution', 0.315751314163208],
 ['academic', 0.307522714138031],
 ['postsecondary', 0.2963041067123413],
 ['education', 0.2874205708503723],
 ['associate', 0.2682255506515503]]

Insert the links between the topics and the text into the db

In [7]:
from embedding import store_cluster_link_entry
d = topic_model.get_document_info(texts)
d_doc = d['Document']
d_name = d['Name']
print(len(d_doc))
print(len(d_name))

s_doc = d_doc.tolist()
s_name = d_name.tolist()
for i, doc in enumerate(s_doc):
    store_cluster_link_entry(db_name, doc, s_name[i])


ValueError: All arrays must be of the same length

In [4]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4179,-1_or_of_the_any,"[or, of, the, any, to, in, and, section, this,...",[The board may take action against any license...
1,0,290,0_racing_horse_wagering_meeting,"[racing, horse, wagering, meeting, races, thor...",[The total percentage deducted from wagers at ...
2,1,243,1_pharmacy_pharmacist_drug_drugs,"[pharmacy, pharmacist, drug, drugs, prescripti...",[(a) A pharmacy located in the state may provi...
3,2,200,2_bar_state_court_attorney,"[bar, state, court, attorney, legal, supreme, ...",[(a) In addition to any criminal penalties pur...
4,3,174,3_lien_owner_vehicle_lienholder,"[lien, owner, vehicle, lienholder, claim, work...",[A petition for a release order shall be verif...
...,...,...,...,...,...
270,269,11,269_solar_energy_system_installation,"[solar, energy, system, installation, systems,...",[(a) When reviewing a request to install a sol...
271,270,11,270_loan_rate_reverse_mortgage,"[loan, rate, reverse, mortgage, you, lender, i...",[(a) No increase in interest provided for in a...
272,271,11,271_hemodialysis_dialysis_training_technician,"[hemodialysis, dialysis, training, technician,...",[(a) Except during training under immediate su...
273,272,11,272_viticultural_appellation_wine_origin,"[viticultural, appellation, wine, origin, napa...",[(a) (1) The Legislature finds and declares th...


Chunking Dataset

In [5]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
import timeit
from datetime import timedelta
from embedding import fetch_entries_with_embeddings_specific_chunk, store_cluster_link_entry, insert_label
total_start_time = timeit.default_timer()
db_name = "law_database.db"
emedding_model = "all-MiniLM-L6-v2"
model_uuid = "1bb4b5f7-1d48-4e18-b44e-e3b4ebe66a10"
chunk_size = 25000
chunk, num_chunks = fetch_entries_with_embeddings_specific_chunk(db_name, model_uuid, chunk_size, 1)
print(num_chunks)
representation_model = KeyBERTInspired()
base_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk[0], embeddings=chunk[3])

topic_labels = base_model.generate_topic_labels(nr_words=3, topic_prefix=False)
print(topic_labels)

d = base_model.get_document_info(chunk[0])
d_doc = d['Document'].tolist()
d_topic_id = d['Topic'].tolist()

for i, doc in enumerate(d_doc):
    store_cluster_link_entry(db_name, doc, topic_labels[d_topic_id[i]], chunk[1][i], chunk[2][i])

for i in range(2, num_chunks + 1):
    start_time = timeit.default_timer()
    print(i)
    chunk_data, s = fetch_entries_with_embeddings_specific_chunk(db_name, model_uuid, chunk_size, i)
    new_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk_data[0], embeddings=chunk_data[3])
    updated_model = BERTopic.merge_models([base_model, new_model])

    # Let's print the newly discover topics
    nr_new_topics = len(set(updated_model.topics_)) - len(set(base_model.topics_))
    new_topics = list(updated_model.topic_labels_.values())[-nr_new_topics:]
    print("The following topics are newly found:")
    print(f"{new_topics}\n")

    topic_labels = new_model.generate_topic_labels(nr_words=3, topic_prefix=False)

    d = new_model.get_document_info(chunk_data[0])
    d_doc = d['Document'].tolist()
    d_topic_id = d['Topic'].tolist()

    for x, doc in enumerate(d_doc):
        store_cluster_link_entry(db_name, doc, topic_labels[d_topic_id[x]], chunk_data[1][x], chunk_data[2][x])
    
    base_model = updated_model

    end_time = timeit.default_timer()
    iteration_time = end_time - start_time
    total_time_so_far = end_time - total_start_time
    average_time_per_iteration = total_time_so_far / i
    estimated_total_time = average_time_per_iteration * num_chunks
    percent_complete = (i / num_chunks) * 100

    print(f"Iteration {i} took {timedelta(seconds=iteration_time)}.")
    print(f"Estimated time remaining: {timedelta(seconds=estimated_total_time)}.")
    print(f"Percent completed: {percent_complete:.2f}%\n")

topic_model = base_model

total_end_time = timeit.default_timer()
total_time_taken = total_end_time - total_start_time
print(f"Total time taken: {timedelta(seconds=total_time_taken)}")

# for i in range(1, num_chunks + 1):
#     chunk_data, s = fetch_entries_with_embeddings_specific_chunk(db_name, chunk_size, i)
#     topics, probs = topic_model.transform(chunk_data[0], embeddings=chunk_data[2])
#     f = new_model.get_topic_info()
#     t = f['Name']
#     topic_names = t.tolist()
#     print(topic_names)
#     for topic_name in topic_names:
#         insert_label(db_name, topic_name)

#     d = new_model.get_document_info(chunk_data[0])
#     d_doc = d['Document']
#     d_name = d['Name']

#     s_doc = d_doc.tolist()
#     s_name = d_name.tolist()
#     for i, doc in enumerate(s_doc):
#         store_cluster_link_entry(db_name, doc, s_name[i])

67


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

['regulations_enforcement_pursuant', 'marriage_therapist_counselor', 'examinations_examination_examiners', 'sec_acc_se', 'pharmacist_pharmacy_intern', 'consumers_consumer_agency', 'lenses_eyecare_prescribing', 'reimbursement_expenses_incurred', 'accredited_accreditation_dentistry', 'expenditures_fiscal_enrollment', 'sterile_pharmacy_pharm', 'pharmacy_rnia_nonresident', 'counties_county_charter', 'supervisory_supervising_assistant', 'justices_presiding_appointed', 'protection_precedence_public', 'executive_duties_delegation', 'cemetery_funeral_crematories', 'restraining_injunction_enforcement', 'bail_arrest_sentencing', 'applicant_documents_notification', 'dental_extended_dentist', 'vocational_nursing_nurses', 'maps_map_referendum', 'optometrists_optometrist_optome', 'diagnostic_illness_symptoms', 'transgender_culturally_ethnicity', 'disputes_dispute_resolving', 'board_boards_medical', 'enforcement_duties_authority', 'dentistry_dentist_dentists', 'documents_fines_fee', 'competency_compl

KeyboardInterrupt: 

In [3]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
import timeit
from datetime import timedelta
from embedding import fetch_entries_with_embeddings_specific_chunk, store_cluster_link_entries_bulk

def process_chunk_and_store_links(db_path, chunk_data, model, model_uuid, representation_model, embedding_model):
    new_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=embedding_model).fit(chunk_data[0], embeddings=chunk_data[3])
    updated_model = BERTopic.merge_models([model, new_model])

    topic_labels = new_model.generate_topic_labels(nr_words=3, topic_prefix=False)
    d = new_model.get_document_info(chunk_data[0])
    d_doc = d['Document'].tolist()
    d_topic_id = d['Topic'].tolist()

    # Prepare entries for bulk insert
    entries_to_insert = [(topic_labels[topic_id], law_entry_uuid, embedding_uuid) for doc, topic_id, law_entry_uuid, embedding_uuid in zip(d_doc, d_topic_id, chunk_data[1], chunk_data[2])]

    # Store entries using the new bulk function
    store_cluster_link_entries_bulk(db_path, entries_to_insert)

    return updated_model

total_start_time = timeit.default_timer()
db_name = "law_database.db"
emedding_model = "all-MiniLM-L6-v2"
model_uuid = "1bb4b5f7-1d48-4e18-b44e-e3b4ebe66a10"
chunk_size = 25000
chunk, num_chunks = fetch_entries_with_embeddings_specific_chunk(db_name, model_uuid, chunk_size, 1)
print(num_chunks)
representation_model = KeyBERTInspired()
base_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk[0], embeddings=chunk[3])

topic_labels = base_model.generate_topic_labels(nr_words=3, topic_prefix=False)
d = base_model.get_document_info(chunk[0])
d_doc = d['Document'].tolist()
d_topic_id = d['Topic'].tolist()

# Prepare entries for bulk insert
entries_to_insert = [(topic_labels[topic_id], law_entry_uuid, embedding_uuid) for doc, topic_id, law_entry_uuid, embedding_uuid in zip(d_doc, d_topic_id, chunk[1], chunk[2])]

# Store entries using the new bulk function
store_cluster_link_entries_bulk(db_name, entries_to_insert)

for i in range(2, num_chunks + 1):
    start_time = timeit.default_timer()
    print(i)
    chunk_data, s = fetch_entries_with_embeddings_specific_chunk(db_name, model_uuid, chunk_size, i)
    base_model = process_chunk_and_store_links(db_name, chunk_data, base_model, model_uuid, representation_model, emedding_model)

    end_time = timeit.default_timer()
    iteration_time = end_time - start_time
    total_time_so_far = end_time - total_start_time
    average_time_per_iteration = total_time_so_far / i
    estimated_total_time = average_time_per_iteration * num_chunks
    percent_complete = (i / num_chunks) * 100

    print(f"Iteration {i} took {timedelta(seconds=iteration_time)}.")
    print(f"Estimated time remaining: {timedelta(seconds=estimated_total_time)}.")
    print(f"Percent completed: {percent_complete:.2f}%\n")

topic_model = base_model

total_end_time = timeit.default_timer()
total_time_taken = total_end_time - total_start_time
print(f"Total time taken: {timedelta(seconds=total_time_taken)}")
topic_model.save("law_database_topic_model", serialization="pickle")

134


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2
Iteration 2 took 0:00:35.377651.
Estimated time remaining: 1:23:15.989791.
Percent completed: 1.49%

3
Iteration 3 took 0:00:29.954078.
Estimated time remaining: 1:17:48.615945.
Percent completed: 2.24%

4
Iteration 4 took 0:00:31.538239.
Estimated time remaining: 1:15:57.996153.
Percent completed: 2.99%

5
Iteration 5 took 0:00:26.770790.
Estimated time remaining: 1:12:43.857921.
Percent completed: 3.73%

6
Iteration 6 took 0:00:33.719471.
Estimated time remaining: 1:13:09.618993.
Percent completed: 4.48%

7
Iteration 7 took 0:00:31.611510.
Estimated time remaining: 1:12:47.667075.
Percent completed: 5.22%

8
Iteration 8 took 0:00:36.166958.
Estimated time remaining: 1:13:47.507011.
Percent completed: 5.97%

9
Iteration 9 took 0:00:30.273989.
Estimated time remaining: 1:13:06.309288.
Percent completed: 6.72%

10
Iteration 10 took 0:00:24.578813.
Estimated time remaining: 1:11:17.036013.
Percent completed: 7.46%

11
Iteration 11 took 0:00:27.490727.
Estimated time remaining: 1:10:23.

Create a hierarchical map of topics

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model_cluster = BERTopic(hdbscan_model=hdbscan_model)

from embedding import fetch_entries
db_name = "law_database.db"
texts, uuids = fetch_entries(db_name)
topics_cluster, probs_cluster = topic_model_cluster.fit_transform(texts)

topic_model_cluster.get_topic_info()

Create UMAP projection form hierarchical map of topics

In [ ]:
from embedding import fetch_entries
db_name = "law_database.db"
docs, uuids = fetch_entries(db_name)
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

# Train BERTopic
topic_model = BERTopic().fit(docs, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
print(reduced_embeddings)


Encode the UMAP projection into a JSON file 
- umap_x
- umap_y
- topic
- document

In [ ]:
import json
print(len(topic_model.get_document_info(docs)))
print(len(reduced_embeddings))

document_info = topic_model.get_document_info(docs)
topics = document_info['Name']
documents = document_info['Document']

# Assuming `reduced_embeddings` is your UMAP data and it's a list of lists or a NumPy array
data = [{"umap_x": float(point[0]), "umap_y": float(point[1]), "topic": topic, "document": document} for point, topic, document in zip(reduced_embeddings, topics, documents)]

# Save the data as a JSON file
with open("umap-data.json", "w") as f:
    json.dump(data, f)

Encode the UMAP projection into JSON file with linked text_uuid
- umap_x
- umap_y
- item_id

In [ ]:
import json

# Get the document information from BERTopic
d = topic_model.get_document_info(texts)
d_doc = d['Document']
d_name = d['Name']

# Convert to list if necessary
s_doc = d_doc.tolist() if hasattr(d_doc, 'tolist') else d_doc
s_name = d_name.tolist() if hasattr(d_name, 'tolist') else d_name

# Create a list of dictionaries for JSON output
data = []
for i, doc in enumerate(s_doc):
    # Use the UUID corresponding to the document's text
    law_entry_uuid = uuids[i]
    data.append({
        "umap_x": float(reduced_embeddings[i][0]),
        "umap_y": float(reduced_embeddings[i][1]),
        "item_id": law_entry_uuid  # Add the law_entry_uuid here
    })

# Save the data as a JSON file
with open("umap-data.json", "w") as f:
    json.dump(data, f)

Future Work:
Update Topic Representation after Training
https://maartengr.github.io/BERTopic/getting_started/topicrepresentation/topicrepresentation.html

Online Topic Modeling with River:
https://maartengr.github.io/BERTopic/getting_started/online/online.html

Multi Aspect Representations:
https://maartengr.github.io/BERTopic/getting_started/multiaspect/multiaspect.html

Zero Shot Topic Modeling
https://maartengr.github.io/BERTopic/getting_started/zeroshot/zeroshot.html

Partial Labels:
https://maartengr.github.io/BERTopic/getting_started/semisupervised/semisupervised.html#partial-labels

In [5]:
from bertopic import BERTopic
from embedding import fetch_entries_with_user_labels
db_name = "law_database.db"
texts, labels, uuids, label_bert_id = fetch_entries_with_user_labels(db_name)

# Print all the values of labels and their uuids where the label is not equal to -1
for label, uuid, label_bert_id in zip(labels, uuids, label_bert_id):
    if label_bert_id != -1:
        print(f"Label: {label}, UUID: {uuid}")

if all(item == -1 for item in label_bert_id):
    print("error no user labels")
else:
    topic_model = BERTopic()
    topics, probs = topic_model.fit_transform(texts, y=label_bert_id)

TypeError: 'int' object is not iterable

In [1]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
import timeit
from datetime import timedelta
from embedding import fetch_entries_with_user_labels_and_embeddings_chunk, store_cluster_link_entry, insert_label
total_start_time = timeit.default_timer()
db_name = "law_database.db"
emedding_model = "all-MiniLM-L6-v2"
chunk_size = 5000
### chunk := texts, labels, uuids, label_bert_id, embeddings
chunk, num_chunks = fetch_entries_with_user_labels_and_embeddings_chunk(db_name, model_uuid, chunk_size, 1)
print(num_chunks)
representation_model = KeyBERTInspired()
base_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk[0], embeddings=chunk[5], y=chunk[4])

topic_labels = base_model.generate_topic_labels(nr_words=3, topic_prefix=False)
print(topic_labels)

d = base_model.get_document_info(chunk[0])
d_doc = d['Document'].tolist()
d_topic_id = d['Topic'].tolist()

for i, doc in enumerate(d_doc):
    store_cluster_link_entry(db_name, doc, topic_labels[d_topic_id[i]], chunk[1][i], chunk[2][i])

for i in range(2, num_chunks + 1):
    start_time = timeit.default_timer()
    print(i)
    chunk_data, s = fetch_entries_with_user_labels_and_embeddings_chunk(db_name, model_uuid, chunk_size, i)
    new_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk_data[0], embeddings=chunk_data[5], y=chunk[4])
    updated_model = BERTopic.merge_models([base_model, new_model])

    # Let's print the newly discover topics
    nr_new_topics = len(set(updated_model.topics_)) - len(set(base_model.topics_))
    new_topics = list(updated_model.topic_labels_.values())[-nr_new_topics:]
    print("The following topics are newly found:")
    print(f"{new_topics}\n")

    topic_labels = new_model.generate_topic_labels(nr_words=3, topic_prefix=False)

    d = new_model.get_document_info(chunk_data[0])
    d_doc = d['Document'].tolist()
    d_topic_id = d['Topic'].tolist()
    
    for x, doc in enumerate(d_doc):
        store_cluster_link_entry(db_name, doc, topic_labels[d_topic_id[x]], chunk_data[1][x], chunk_data[2][x])
    
    base_model = updated_model

    end_time = timeit.default_timer()
    iteration_time = end_time - start_time
    total_time_so_far = end_time - total_start_time
    average_time_per_iteration = total_time_so_far / i
    estimated_total_time = average_time_per_iteration * num_chunks
    percent_complete = (i / num_chunks) * 100

    print(f"Iteration {i} took {timedelta(seconds=iteration_time)}.")
    print(f"Estimated time remaining: {timedelta(seconds=estimated_total_time)}.")
    print(f"Percent completed: {percent_complete:.2f}%\n")

topic_model = base_model

total_end_time = timeit.default_timer()
total_time_taken = total_end_time - total_start_time
print(f"Total time taken: {timedelta(seconds=total_time_taken)}")

[nltk_data] Downloading package punkt to /home/luc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1


AttributeError: 'tuple' object has no attribute 'get_topic_info'

In [4]:
# topic_model.get_topic_info()

base_model.get_topics()

AttributeError: 'numpy.ndarray' object has no attribute 'get_topics'

LLM generated labels:
https://maartengr.github.io/BERTopic/getting_started/representation/llm#zephyr-mistral-7b

In [ ]:
### DRINK ME
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
import timeit
from datetime import timedelta
from embedding import fetch_entries_with_user_labels_and_embeddings_chunk, store_cluster_link_entries_bulk

def process_chunk_and_store_links(db_path, chunk_data, model, model_uuid, representation_model, embedding_model):
    new_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk_data[0], embeddings=chunk_data[4], y=chunk[3])
    updated_model = BERTopic.merge_models([model, new_model])

    topic_labels = new_model.generate_topic_labels(nr_words=3, topic_prefix=False)
    d = new_model.get_document_info(chunk_data[0])
    d_doc = d['Document'].tolist()
    d_topic_id = d['Topic'].tolist()

    # Let's print the newly discover topics
    new_topics = list(updated_model.topic_labels_.values())
    print("The following topics are newly found:")
    print(f"{new_topics}\n")

    # Prepare entries for bulk insert
    entries_to_insert = [(topic_labels[topic_id], law_entry_uuid) for doc, topic_id, law_entry_uuid in zip(d_doc, d_topic_id, chunk_data[1])]

    # Store entries using the new bulk function
    store_cluster_link_entries_bulk(db_path, entries_to_insert)

    return updated_model

total_start_time = timeit.default_timer()
db_name = "law_database.db"
emedding_model = "all-MiniLM-L6-v2"
model_uuid = "1bb4b5f7-1d48-4e18-b44e-e3b4ebe66a10"
chunk_size = 25000
### chunk := texts, labels, uuids, label_bert_id, embeddings, embedding_id
chunk, num_chunks = fetch_entries_with_user_labels_and_embeddings_chunk(db_name, model_uuid, chunk_size, 1)
print(num_chunks)
representation_model = KeyBERTInspired()
base_model = BERTopic(representation_model=representation_model, min_topic_size=15, embedding_model=emedding_model).fit(chunk[0], embeddings=chunk[4], y=chunk[3])

topic_labels = base_model.generate_topic_labels(nr_words=3, topic_prefix=False)
d = base_model.get_document_info(chunk[0])
d_doc = d['Document'].tolist()
d_topic_id = d['Topic'].tolist()

# Prepare entries for bulk insert
entries_to_insert = [(topic_labels[topic_id], law_entry_uuid) for doc, topic_id, law_entry_uuid in zip(d_doc, d_topic_id, chunk[1])]

# Store entries using the new bulk function
store_cluster_link_entries_bulk(db_name, entries_to_insert)

for i in range(2, num_chunks + 1):
    start_time = timeit.default_timer()
    print(i)
    chunk_data, s = fetch_entries_with_user_labels_and_embeddings_chunk(db_name, model_uuid, chunk_size, i)
    base_model = process_chunk_and_store_links(db_name, chunk_data, base_model, model_uuid, representation_model, emedding_model)

    end_time = timeit.default_timer()
    iteration_time = end_time - start_time
    total_time_so_far = end_time - total_start_time
    average_time_per_iteration = total_time_so_far / i
    estimated_total_time = average_time_per_iteration * num_chunks
    percent_complete = (i / num_chunks) * 100

    print(f"Iteration {i} took {timedelta(seconds=iteration_time)}.")
    print(f"Estimated time remaining: {timedelta(seconds=estimated_total_time)}.")
    print(f"Percent completed: {percent_complete:.2f}%\n")

topic_model = base_model

total_end_time = timeit.default_timer()
total_time_taken = total_end_time - total_start_time
print(f"Total time taken: {timedelta(seconds=total_time_taken)}")
topic_model.save("law_database_topic_model_user_labels", serialization="pickle")

In [9]:
%pip install ctransformers
%pip install --upgrade git+https://github.com/huggingface/transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cbh8_jpz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cbh8_jpz
  Resolved https://github.com/huggingface/transformers to commit 5b6fa2306add0cb06dd1a0ecd708633e8c7e5e58
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8460817 sha256=225eaa2640b16edeeaafb5bc30f638876db5f8bcaa96fa211dfd1c3fe0c574cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-zg36e0fq/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
Note: you may need to restart the kernel

In [1]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from bertopic.representation import TextGeneration, KeyBERTInspired 
from bertopic import BERTopic
from embedding import fetch_entries_with_embeddings_specific_chunk, store_cluster_link_entry, insert_label

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

prompt = """<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.</s>
<|assistant|>"""

# Text generation with Zephyr
zephyr = TextGeneration(generator, prompt=prompt)
representation_model = {"Zephyr": zephyr}

# Topic Modeling
# representation_model = KeyBERTInspired(representation_model=representation_model, verbose=True)

db_name = "law_database.db"
emedding_model = "all-MiniLM-L6-v2"
chunk_size = 5000
chunk, num_chunks = fetch_entries_with_embeddings_specific_chunk(db_name, chunk_size, 1)
print(num_chunks)
# representation_model = KeyBERTInspired()
base_model = BERTopic(representation_model=representation_model, embedding_model=emedding_model, verbose=True).fit(chunk[0], embeddings=chunk[2])
topic_model = base_model
topics, probs = topic_model.transform(chunk[0], embeddings=chunk[2])

print(topics)

[nltk_data] Downloading package punkt to /home/luc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

127


2024-02-15 08:14:00,338 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-15 08:14:19,627 - BERTopic - Dimensionality - Completed ✓
2024-02-15 08:14:19,628 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-15 08:14:19,803 - BERTopic - Cluster - Completed ✓
2024-02-15 08:14:19,806 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|                                                   | 0/198 [00:00<?, ?it/s]Number of tokens (14537) exceeded maximum context length (512).


In [ ]:
from embedding import fetch_entries
db_name = "law_database.db"
texts, uuids = fetch_entries(db_name)

topics, probs = topic_model.fit_transform(texts)

In [ ]:
topic_model.get_topic_info()
topic_model.get_topics()